In [ ]:
import pandas as pd
import numpy as np
# change file names only
# input file
X = "data_for_youcef.csv"
# output file
Y = "data_final_cleaned.csv"

df = pd.read_csv(X)


# removing false prices

In [ ]:
SEQUENTIAL_TROLLS = {
    123,
    1111,
    321,
    222222,
    1234,
    12345,
    123456,
    1234567,
    12345678,
    8976378,
    5649841,
    123456789,
    1223789,
}


# Minimum number of repeated digits to flag (e.g. 3 catches 111, 999, 333 …)
MIN_REPEAT_LENGTH = 3


def is_troll_price(price):
    """Return True if the price is a troll value."""
    if pd.isna(price):
        return False

    p = int(price)
    s = str(p)

    # 1) Repeated single digit: 111, 999, 4444, 111111, 222222222 …
    if len(set(s)) == 1 and len(s) >= MIN_REPEAT_LENGTH:
        return True

    # 2) Sequential digits: 123, 1234, 12345, 123456, 1234567 …
    if p in SEQUENTIAL_TROLLS:
        return True

    return False


df["is_troll_price"] = df["price_preview"].apply(is_troll_price)
# set estimated_price_dzd to NaN where is_troll_price is True
df.loc[df["is_troll_price"], "estimated_price_dzd"] = np.nan
# remove is_troll_price column
df.drop(columns=["is_troll_price"], inplace=True)

### Predicting the correct price based on the market value of the compenents 

In [34]:
df_cpu_prices = pd.read_csv("cpu_prices.csv")
df_gpu_prices = pd.read_csv("gpu_prices.csv")

cpu_price_map = dict(zip(df_cpu_prices["cpu_name"], df_cpu_prices["estimated_price"]))

gpu_price_map = dict(zip(df_gpu_prices["gpu_name"], df_gpu_prices["estimated_price"]))

In [10]:

for  row in cpu_price_map.items():
    cpu_price = cpu_price_map.get(row[0], 15000)
    print (cpu_price)

25000
65000
55000
45000
12000
32000
42000
35000
85000
35000
35000
35000
40000
32000
25000
65000
25000
20000
22000
25000
55000
110000
30000
32000
22000
6000
15000
22000
18000
15000
25000
15000
180000
65000
25000
30000
5500
5000
5000
15000
25000
6000
120000
55000
55000
85000
220000
50000
100000
9000
180000
32000
35000
180000
180000
250000
55000
120000
65000
50000
200000
55000
170000
150000
50000
110000
110000
30000
200000
150000
130000
40000
110000
150000
160000
100000
18000
150000
25000
30000
150000
150000
110000
65000
150000
15000
90000
150000
55000
30000
120000
180000
110000
140000
140000
110000
65000
12000
160000
90000
90000
150000
65000
130000
20000
120000
130000
70000
150000
50000
150000
85000
50000
85000
95000
55000
40000
130000
85000
110000
80000
140000
65000
90000
95000
80000
80000
90000
120000
120000
110000
55000
45000
120000
85000
80000
85000
110000
60000
55000
110000
85000
85000
65000
110000
110000
100000
55000
110000
65000
100000
110000
95000
110000
55000
15000
90000
65000
1

In [ ]:
def estimate_ram_price(ram_gb):
    if ram_gb <= 4:
        return 4000
    elif ram_gb <= 8:
        return 8000
    elif ram_gb <= 16:
        return 15000
    elif ram_gb <= 32:
        return 28000
    else:
        return 40000


def estimate_storage_price(ssd_gb, hdd_gb):
    price = 0

    # SSD
    if ssd_gb > 0:
        price += (ssd_gb / 256) * 8000  # 256GB ≈ 8k DZD

    # HDD
    if hdd_gb > 0:
        price += (hdd_gb / 1000) * 6000  # 1TB ≈ 6k DZD

    return price


BRAND_MULTIPLIER = {
    "ROG": 1.25,  # ASUS ROG (Premium gaming)
    "ALIENWARE": 1.25,  # Dell Alienware (Premium gaming)
    "STEALTH": 1.20,  # MSI Stealth (Premium)
    "RAZER": 1.30,  # Razer (Ultra premium)
    "MAC": 1.50,  # MacBook (Apple premium)
    "THINKPAD": 1.15,  # Lenovo ThinkPad (Business premium)
    "VECTOR": 1.20,  # MSI Vector (Gaming)
    "ZENBOOK": 1.15,  # ASUS ZenBook (Premium ultrabook)
    "PRECISION": 1.20,  # Dell Precision (Workstation)
    "TUF": 1.05,  # ASUS TUF (Value gaming)
    "KATANA": 1.05,  # MSI Katana (Value gaming)
    "VIVOBOOK": 1.00,  # ASUS VivoBook (Standard)
    "IDEAPAD": 0.95,  # Lenovo IdeaPad (Budget)
    "INSPIRON": 0.95,  # Dell Inspiron (Budget)
    "PAVILION": 0.95,  # HP Pavilion (Budget)
    "ASPIRE": 0.90,  # Acer Aspire (Budget)
}

RAM_PRICE_PER_GB = {
    "DDR5": 2000,  # DDR5 (Latest, most expensive)
    "DDR5X": 2200,  # DDR5X (Premium)
    "LPDDR5X": 2400,  # LPDDR5X (High-end laptops)
    "LPDDR5": 2100,  # LPDDR5
    "DDR4": 1500,  # DDR4 (Common)
    "DDR4X": 1700,  # DDR4X
    "DDR3": 1000,  # DDR3 (Older)
    "DDR2": 800,  # DDR2 (Legacy)
    "DEFAULT": 1500,  # Default if type unknown
}


def parse_ram_size(ram_str):
    """Convert RAM size string to GB (numeric)"""
    if pd.isna(ram_str) or ram_str == "":
        return 0

    ram_str = str(ram_str).upper().strip()

    # Extract numeric value
    numeric_part = "".join(filter(lambda x: x.isdigit() or x == ".", ram_str))
    if not numeric_part:
        return 0

    value = float(numeric_part)

    # Convert to GB
    if "MB" in ram_str:
        return value / 1024
    elif "GB" in ram_str:
        return value
    else:
        return value


def parse_storage_size(size_str):
    """Convert storage size string to GB (numeric)"""
    if pd.isna(size_str) or size_str == "":
        return 0

    size_str = str(size_str).upper().strip()

    # Handle combined storage (e.g., "1TB + 512GB")
    if "+" in size_str:
        parts = size_str.split("+")
        total = 0
        for part in parts:
            total += parse_storage_size(part.strip())
        return total

    # Extract numeric value
    numeric_part = "".join(filter(lambda x: x.isdigit() or x == ".", size_str))
    if not numeric_part:
        return 0

    value = float(numeric_part)

    # Convert to GB
    if "TB" in size_str:
        return value * 1024
    elif "GB" in size_str:
        return value
    elif "MB" in size_str:
        return value / 1024
    else:
        return value


def brand_multiplier(model_name):
    for brand, mult in BRAND_MULTIPLIER.items():
        if brand.lower() in str(model_name).lower():
            return mult
    return 1.0


def get_ram_price(ram_size_gb, ram_type):
    """Calculate RAM price based on size and type"""
    if ram_size_gb == 0 or ram_size_gb > 128:  # Cap at 128GB to avoid errors
        ram_size_gb = (
            min(ram_size_gb, 128) if ram_size_gb > 0 else 8
        )  # Default to 8GB if 0

    # Determine RAM type price
    if pd.isna(ram_type):
        price_per_gb = RAM_PRICE_PER_GB["DEFAULT"]
    else:
        ram_type = str(ram_type).upper().strip()
        price_per_gb = RAM_PRICE_PER_GB.get(ram_type, RAM_PRICE_PER_GB["DEFAULT"])

    return ram_size_gb * price_per_gb


def estimate_price(row):
    # CPU
    cpu_price = cpu_price_map.get(row["mapped_cpu_name"], 15000)

    # GPU
    gpu_price = gpu_price_map.get(row["gpu_name"], 0)

    # RAM
    ram_price = get_ram_price(parse_ram_size(row["RAM_SIZE"]), row["RAM_TYPE"])

    # Storage
    storage_price = estimate_storage_price(
        parse_storage_size(row["SSD_SIZE"]), parse_storage_size(row["HDD_SIZE"])
    )

    base_price = cpu_price + gpu_price + ram_price + storage_price

    # Brand multiplier
    final_price = base_price * brand_multiplier(row["model_name"])

    return round(final_price, -2)  # round to nearest 100 DZD


def fix_price_scale(real_price, estimated_price):
    if estimated_price == 0 or real_price == 0:
        return real_price

    ratio = real_price / estimated_price

    while ratio >= 8:
        real_price = real_price / 10
        ratio = real_price / estimated_price
    while ratio <= 1 / 8:
        real_price = real_price * 10
        ratio = real_price / estimated_price
    return real_price


In [23]:
df["estimated_price_dzd"] = df.apply(estimate_price, axis=1)

df["price_corrected"] = df.apply(
    lambda row: fix_price_scale(row["price_preview"], row["estimated_price_dzd"]),
    axis=1,
)

df["price_multiplier_diff"] = (df["price_preview"] / df["estimated_price_dzd"]).round(2)
# remove the estimated_prices 
df.drop(columns=["estimated_price_dzd"], inplace=True)
# remove price_multiplier_diff 
df.drop(columns=["price_multiplier_diff"], inplace=True)

In [37]:
df.to_csv(Y, index=False)